# Economic Inequality in Latin America

### Trends in the Gini Index and its link to Average Income.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import plotly.express as px
from matplotlib.patches import Patch
from matplotlib.lines import Line2D



### EDA

In [ ]:
# Exploring .csv files from https://data.worldbank.org/indicator/SI.POV.GINI?type=points&year=2019 

df_1 = pd.read_csv("API_SI.POV.GINI_DS2_en_csv_v2_19376.csv",
                   delimiter=',',
                    skiprows=4,
                     engine='python')

df_1

In [ ]:
df_2 = pd.read_csv("Metadata_Country_API_SI.POV.GINI_DS2_en_csv_v2_19376.csv")
df_2

In [ ]:
df_3 = pd.read_csv("Metadata_Indicator_API_SI.POV.GINI_DS2_en_csv_v2_19376.csv")
df_3

In [ ]:
# Extract column 'Latin America & Caribbean' from the dataframe 'df_2'
df_2[df_2['Region'] == 'Latin America & Caribbean']

In [ ]:
latinamerican_countries = df_2[df_2['Region'] == 'Latin America & Caribbean'] 

In [ ]:
latinamerican_countries_by_income = latinamerican_countries.groupby('IncomeGroup')

In [ ]:
latinamerican_countries_by_income.size()

In [ ]:
latinamerican_income_distribution = latinamerican_countries['IncomeGroup'].value_counts().to_dict()
latinamerican_income_distribution

#hacer un plot de esto (Income per region)

### Data Visualization

In [ ]:
# Plot latinoamerican income distribution
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

income_categories = list(latinamerican_income_distribution.keys())
country_counts = list(latinamerican_income_distribution.values())

colors = ['#9c6793', '#1170aa', '#57606c' ]

bars = plt.bar(income_categories, country_counts, color= colors)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height}', ha='center', va='bottom')
    plt.title('Latin American Income Distribution', fontsize=15)
    
plt.xlabel('Income Group', fontsize=12)
plt.ylabel('Number of countries', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
latinamerican_countries_by_income['TableName'].apply(list)

In [ ]:
latinamerican_countries_by_income_to_dict = latinamerican_countries_by_income['TableName'].apply(list).to_dict()
latinamerican_countries_by_income_to_dict

In [ ]:
latam_high_income = latinamerican_countries_by_income_to_dict['High income']
latam_upper_middle_income = latinamerican_countries_by_income_to_dict['Upper middle income']
latam_lower_middle_income = latinamerican_countries_by_income_to_dict['Lower middle income']

In [42]:
latam_high_income

['Aruba',
 'Antigua and Barbuda',
 'Bahamas, The',
 'Barbados',
 'Chile',
 'Curaçao',
 'Cayman Islands',
 'Guyana',
 'St. Kitts and Nevis',
 'St. Martin (French part)',
 'Panama',
 'Puerto Rico',
 'Sint Maarten (Dutch part)',
 'Turks and Caicos Islands',
 'Trinidad and Tobago',
 'Uruguay',
 'British Virgin Islands',
 'Virgin Islands (U.S.)']

In [40]:
type(latam_high_income)

list

In [ ]:
latam_income_by_group = [latam_high_income, latam_upper_middle_income, latam_lower_middle_income]


In [41]:
type(latam_income_by_group)

str

In [ ]:
# Plotting Latino American income groups on the world map

world = gpd.read_file('https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip')

#Checking columns from the dataframe 'world'
print(world.columns)
# Detecting column of the code country
iso_column = 'ISO_A3'

# merging data for plot
merged_data = world.merge(latinamerican_countries, 
                          left_on=iso_column, 
                          right_on='Country Code', 
                          how='left')
# Subset data to Latin America and the Caribbean 
latam = merged_data[merged_data['Region'] == 'Latin America & Caribbean']

# Plot the world as a light gray background 
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

world.plot(ax=ax, color='lightgray', edgecolor='white', linewidth=0.5)

# Visualize Latin American countries, color-coded by income group
colors = {
    'High income': '#1170aa',       # Azul fuerte
    'Upper middle income': '#9c6793', # ciruela claro
    'Lower middle income': '#57606c'  # Gris oscuro
}
# Check if countries belong to the income group and plot only those that do
for income_group, color in colors.items():
    countries_in_group = latam[latam['IncomeGroup'] == income_group]
    if not countries_in_group.empty:
        countries_in_group.plot(
            ax=ax, 
            color=color, 
            edgecolor='white', 
            linewidth=0.8
        )

# Customizing plot
ax.set_title('Income Stratification in Latin America and the Caribbean', fontsize=16)
ax.set_axis_off()

legend_elements = [
    Patch(facecolor='#1170aa', edgecolor='white', label='High Income'),
    Patch(facecolor='#9c6793', edgecolor='white', label='Upper Middle Income'),
    Patch(facecolor='#57606c', edgecolor='white', label='Lower Middle Income')
]
ax.legend(handles=legend_elements, loc='lower left', frameon=True, framealpha=0.9)

plt.tight_layout()
plt.savefig('latinamerica_income_map.png', dpi=300, bbox_inches='tight')
plt.show()